In [1]:
import pandas as pd
import re

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
x=pd.read_csv('Substitute_meat_Combined.csv')

In [5]:
x.drop('Unnamed: 0',inplace=True,axis=1)

In [6]:
x.columns

Index(['Geography', 'Time', 'Product', 'UPC 13 digit', 'Unit Sales',
       'Unit Sales No Merch', 'Unit Sales Any Merch',
       'Unit Sales Price Reductions Only', 'Unit Sales Feature Only',
       'Unit Sales Display Only', 'Unit Sales Feature and Display',
       'Volume Sales', 'Volume Sales No Merch', 'Volume Sales Any Merch',
       'Volume Sales Price Reductions Only', 'Volume Sales Feature Only',
       'Volume Sales Display Only', 'Volume Sales Feature and Display',
       'Dollar Sales', 'Dollar Sales No Merch', 'Dollar Sales Any Merch',
       'Dollar Sales Price Reductions Only', 'Dollar Sales Feature Only',
       'Dollar Sales Display Only', 'Dollar Sales Feature and Display',
       'Price per Unit', 'Price per Unit No Merch', 'Price per Unit Any Merch',
       'Price per Unit Price Reductions Only', 'Price per Unit Feature Only',
       'Price per Unit Display Only', 'Price per Unit Feature and Display',
       'Price per Volume', 'Price per Volume No Merch',
       'P

In [9]:
p=x[['Geography','Unit Sales','Time','Product','Unit Sales No Merch', 'Unit Sales Any Merch',
       'Unit Sales Price Reductions Only','Price per Unit', 'Price per Unit No Merch', 'Price per Unit Any Merch',
       'Price per Unit Price Reductions Only','year']]

In [11]:
p = p[p['Geography'] != 'Total US - Multi Outlet + Conv']

In [13]:
p.drop('Time',axis=1,inplace=True)

In [15]:
df= p['Product']
df=df.to_frame()

In [17]:
def extract_info(row):
    pattern = re.compile(r'^(?P<Company>[^\s]+)\s(?P<Product_Type>.+?)\s-\s(?P<UPCID>\d+)$')
    match = pattern.match(row['Product'])
    if match:
        return pd.Series([match.group('Company'), match.group('Product_Type'), match.group('UPCID')])
    return pd.Series([None, None, None])

# Apply the function to the DataFrame, creating new columns
df[['Company', 'Product', 'UPC ID']] = df.apply(extract_info, axis=1)

In [18]:
df.drop('UPC ID',axis=1,inplace=True)
df.drop('Company',axis=1,inplace=True)

In [19]:
# List of values to check against
values_list = ['REGULAR', 'CHICKEN', 'ORIGINAL','BLACK BEAN','ITALIAN','SAUSAGE','TERIYAKI','SWEET AND SOUR','SZECHUAN']

# Define the default value for non-matching cases
default_value = 'REGULAR'
regular=[]
# Function to check for matches and return only the match or default value
def check_for_match(description, values_list):
    for value in values_list:
        if value in description:
            return value
    regular.append(description)
    return default_value

# Apply the function to the 'Description' column to create a new 'Match' column
df['Flavor'] = df['Product'].apply(check_for_match, args=([values_list]))

In [20]:
df.drop('Product',inplace=True,axis=1)

In [21]:
x=pd.concat([p,df],axis=1)

In [22]:
x

,Geography,Unit Sales,Product,Unit Sales No Merch,Unit Sales Any Merch,Unit Sales Price Reductions Only,Price per Unit,Price per Unit No Merch,Price per Unit Any Merch,Price per Unit Price Reductions Only,year,Flavor
36805,California - Standard - Multi Outlet + Conv,232.985395,AMYS FROZEN BLACK BEAN MEAT SUBSTITUTE BURGER ...,229.985395,3.000000,3.000000,6.864690,6.889274,4.980000,4.980000,2020,BLACK BEAN
36806,California - Standard - Multi Outlet + Conv,313.706756,AMYS FROZEN CALIFORNIA VEGGIE MEAT SUBSTITUTE ...,302.807146,10.899610,7.039762,6.522935,6.555666,5.613643,5.407289,2020,REGULAR
36807,California - Standard - Multi Outlet + Conv,283.551283,AMYS FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE B...,279.691435,3.859848,53.850315,6.376719,6.382056,5.990000,4.179696,2020,REGULAR
36808,California - Standard - Multi Outlet + Conv,409.947925,AMYS FROZEN REGULAR MEAT SUBSTITUTE BURGER 16 ...,403.515004,6.432921,3.859689,8.129029,8.126399,8.294005,8.096667,2020,REGULAR
36809,California - Standard - Multi Outlet + Conv,273.429006,AMYS FROZEN REGULAR MEAT SUBSTITUTE LINK 6 OZ ...,266.691106,6.737900,4.164668,6.694234,6.718803,5.721792,5.432500,2020,REGULAR
...,...,...,...,...,...,...,...,...,...,...,...,...
991775,West - Standard - Multi Outlet + Conv,23.790567,UPTONS NATURALS REFRIGERATED REGULAR MEAT SUBS...,23.790567,69.041003,53.850315,6.513050,6.513050,4.282196,4.179696,2024,REGULAR
991776,West - Standard - Multi Outlet + Conv,107.773600,UPTONS NATURALS REFRIGERATED TRADITIONAL SEITA...,73.564285,34.209315,34.209315,5.942539,6.545713,4.645463,4.645463,2024,REGULAR
991777,West - Standard - Multi Outlet + Conv,1240.627818,FRANKLIN FARMS REFRIGERATED CHICKEN PLANT BASE...,1110.825151,129.802667,100.438719,6.160735,6.155361,6.206728,6.174725,2024,CHICKEN
991778,West - Standard - Multi Outlet + Conv,39.020600,LIGHTLIFE SMART DELI REFRIGERATED TURKEY POULT...,12.322003,26.698597,26.698597,3.550733,3.682327,3.490000,3.490000,2024,REGULAR


In [23]:
company_list=['GARDEIN','MORNINGSTAR','IMPOSSIBLE','BEYOND MEAT','TOFURKY','FIELD ROAST','DR PRAEGERS','BOCA','PRIVATE LABEL']
c= p['Product'].to_frame()
#c[['Company', 'Product', 'UPC ID']] = c.apply(extract_info, axis=1)
#c=c['Product'].to_frame()
default_value = 'OTHERS'
c['Brand'] = c['Product'].apply(check_for_match, args=([company_list]))

In [24]:
c=c['Brand'].to_frame()

In [25]:
x=pd.concat([x,c],axis=1)

In [26]:
x

,Geography,Unit Sales,Product,Unit Sales No Merch,Unit Sales Any Merch,Unit Sales Price Reductions Only,Price per Unit,Price per Unit No Merch,Price per Unit Any Merch,Price per Unit Price Reductions Only,year,Flavor,Brand
36805,California - Standard - Multi Outlet + Conv,232.985395,AMYS FROZEN BLACK BEAN MEAT SUBSTITUTE BURGER ...,229.985395,3.000000,3.000000,6.864690,6.889274,4.980000,4.980000,2020,BLACK BEAN,OTHERS
36806,California - Standard - Multi Outlet + Conv,313.706756,AMYS FROZEN CALIFORNIA VEGGIE MEAT SUBSTITUTE ...,302.807146,10.899610,7.039762,6.522935,6.555666,5.613643,5.407289,2020,REGULAR,OTHERS
36807,California - Standard - Multi Outlet + Conv,283.551283,AMYS FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE B...,279.691435,3.859848,53.850315,6.376719,6.382056,5.990000,4.179696,2020,REGULAR,OTHERS
36808,California - Standard - Multi Outlet + Conv,409.947925,AMYS FROZEN REGULAR MEAT SUBSTITUTE BURGER 16 ...,403.515004,6.432921,3.859689,8.129029,8.126399,8.294005,8.096667,2020,REGULAR,OTHERS
36809,California - Standard - Multi Outlet + Conv,273.429006,AMYS FROZEN REGULAR MEAT SUBSTITUTE LINK 6 OZ ...,266.691106,6.737900,4.164668,6.694234,6.718803,5.721792,5.432500,2020,REGULAR,OTHERS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991775,West - Standard - Multi Outlet + Conv,23.790567,UPTONS NATURALS REFRIGERATED REGULAR MEAT SUBS...,23.790567,69.041003,53.850315,6.513050,6.513050,4.282196,4.179696,2024,REGULAR,OTHERS
991776,West - Standard - Multi Outlet + Conv,107.773600,UPTONS NATURALS REFRIGERATED TRADITIONAL SEITA...,73.564285,34.209315,34.209315,5.942539,6.545713,4.645463,4.645463,2024,REGULAR,OTHERS
991777,West - Standard - Multi Outlet + Conv,1240.627818,FRANKLIN FARMS REFRIGERATED CHICKEN PLANT BASE...,1110.825151,129.802667,100.438719,6.160735,6.155361,6.206728,6.174725,2024,CHICKEN,OTHERS
991778,West - Standard - Multi Outlet + Conv,39.020600,LIGHTLIFE SMART DELI REFRIGERATED TURKEY POULT...,12.322003,26.698597,26.698597,3.550733,3.682327,3.490000,3.490000,2024,REGULAR,OTHERS


In [27]:
x = x[x['Brand'] != 'UNKNOWN']

In [28]:
x

,Geography,Unit Sales,Product,Unit Sales No Merch,Unit Sales Any Merch,Unit Sales Price Reductions Only,Price per Unit,Price per Unit No Merch,Price per Unit Any Merch,Price per Unit Price Reductions Only,year,Flavor,Brand
36805,California - Standard - Multi Outlet + Conv,232.985395,AMYS FROZEN BLACK BEAN MEAT SUBSTITUTE BURGER ...,229.985395,3.000000,3.000000,6.864690,6.889274,4.980000,4.980000,2020,BLACK BEAN,OTHERS
36806,California - Standard - Multi Outlet + Conv,313.706756,AMYS FROZEN CALIFORNIA VEGGIE MEAT SUBSTITUTE ...,302.807146,10.899610,7.039762,6.522935,6.555666,5.613643,5.407289,2020,REGULAR,OTHERS
36807,California - Standard - Multi Outlet + Conv,283.551283,AMYS FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE B...,279.691435,3.859848,53.850315,6.376719,6.382056,5.990000,4.179696,2020,REGULAR,OTHERS
36808,California - Standard - Multi Outlet + Conv,409.947925,AMYS FROZEN REGULAR MEAT SUBSTITUTE BURGER 16 ...,403.515004,6.432921,3.859689,8.129029,8.126399,8.294005,8.096667,2020,REGULAR,OTHERS
36809,California - Standard - Multi Outlet + Conv,273.429006,AMYS FROZEN REGULAR MEAT SUBSTITUTE LINK 6 OZ ...,266.691106,6.737900,4.164668,6.694234,6.718803,5.721792,5.432500,2020,REGULAR,OTHERS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991775,West - Standard - Multi Outlet + Conv,23.790567,UPTONS NATURALS REFRIGERATED REGULAR MEAT SUBS...,23.790567,69.041003,53.850315,6.513050,6.513050,4.282196,4.179696,2024,REGULAR,OTHERS
991776,West - Standard - Multi Outlet + Conv,107.773600,UPTONS NATURALS REFRIGERATED TRADITIONAL SEITA...,73.564285,34.209315,34.209315,5.942539,6.545713,4.645463,4.645463,2024,REGULAR,OTHERS
991777,West - Standard - Multi Outlet + Conv,1240.627818,FRANKLIN FARMS REFRIGERATED CHICKEN PLANT BASE...,1110.825151,129.802667,100.438719,6.160735,6.155361,6.206728,6.174725,2024,CHICKEN,OTHERS
991778,West - Standard - Multi Outlet + Conv,39.020600,LIGHTLIFE SMART DELI REFRIGERATED TURKEY POULT...,12.322003,26.698597,26.698597,3.550733,3.682327,3.490000,3.490000,2024,REGULAR,OTHERS


In [29]:
x['Product% No Merch']=(x['Unit Sales No Merch'] / (x['Unit Sales No Merch'] + x['Unit Sales Any Merch'] + x['Unit Sales Price Reductions Only'])) 

In [30]:
x['Product% Any Merch']=(x['Unit Sales Any Merch'] / (x['Unit Sales No Merch'] + x['Unit Sales Any Merch'] + x['Unit Sales Price Reductions Only'])) 

In [31]:
x['Price Change %'] = (x['Price per Unit Any Merch'] - x['Price per Unit No Merch'])/x['Price per Unit No Merch']

In [32]:
x

,Geography,Unit Sales,Product,Unit Sales No Merch,Unit Sales Any Merch,Unit Sales Price Reductions Only,Price per Unit,Price per Unit No Merch,Price per Unit Any Merch,Price per Unit Price Reductions Only,year,Flavor,Brand,Product% No Merch,Product% Any Merch,Price Change %
36805,California - Standard - Multi Outlet + Conv,232.985395,AMYS FROZEN BLACK BEAN MEAT SUBSTITUTE BURGER ...,229.985395,3.000000,3.000000,6.864690,6.889274,4.980000,4.980000,2020,BLACK BEAN,OTHERS,0.974575,0.012713,-0.277137
36806,California - Standard - Multi Outlet + Conv,313.706756,AMYS FROZEN CALIFORNIA VEGGIE MEAT SUBSTITUTE ...,302.807146,10.899610,7.039762,6.522935,6.555666,5.613643,5.407289,2020,REGULAR,OTHERS,0.944070,0.033982,-0.143696
36807,California - Standard - Multi Outlet + Conv,283.551283,AMYS FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE B...,279.691435,3.859848,53.850315,6.376719,6.382056,5.990000,4.179696,2020,REGULAR,OTHERS,0.828957,0.011440,-0.061431
36808,California - Standard - Multi Outlet + Conv,409.947925,AMYS FROZEN REGULAR MEAT SUBSTITUTE BURGER 16 ...,403.515004,6.432921,3.859689,8.129029,8.126399,8.294005,8.096667,2020,REGULAR,OTHERS,0.975127,0.015546,0.020625
36809,California - Standard - Multi Outlet + Conv,273.429006,AMYS FROZEN REGULAR MEAT SUBSTITUTE LINK 6 OZ ...,266.691106,6.737900,4.164668,6.694234,6.718803,5.721792,5.432500,2020,REGULAR,OTHERS,0.960725,0.024273,-0.148391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991775,West - Standard - Multi Outlet + Conv,23.790567,UPTONS NATURALS REFRIGERATED REGULAR MEAT SUBS...,23.790567,69.041003,53.850315,6.513050,6.513050,4.282196,4.179696,2024,REGULAR,OTHERS,0.162192,0.470685,-0.342521
991776,West - Standard - Multi Outlet + Conv,107.773600,UPTONS NATURALS REFRIGERATED TRADITIONAL SEITA...,73.564285,34.209315,34.209315,5.942539,6.545713,4.645463,4.645463,2024,REGULAR,OTHERS,0.518121,0.240940,-0.290305
991777,West - Standard - Multi Outlet + Conv,1240.627818,FRANKLIN FARMS REFRIGERATED CHICKEN PLANT BASE...,1110.825151,129.802667,100.438719,6.160735,6.155361,6.206728,6.174725,2024,CHICKEN,OTHERS,0.828315,0.096791,0.008345
991778,West - Standard - Multi Outlet + Conv,39.020600,LIGHTLIFE SMART DELI REFRIGERATED TURKEY POULT...,12.322003,26.698597,26.698597,3.550733,3.682327,3.490000,3.490000,2024,REGULAR,OTHERS,0.187495,0.406253,-0.052230


In [33]:
x.to_excel('PREDICTIVE_DATASET.xlsx')

/var/folders/x3/5n0yyb7911zg532jx71xlkrr0000gn/T/ipykernel_43648/577214675.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  x.to_excel('PREDICTIVE_DATASET.xlsx')


KeyboardInterrupt: 

In [28]:
import pandas as pd
x=pd.read_excel('PREDICTIVE_DATASET.xlsx')

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.9' currently installed).

In [37]:
x.columns = x.columns.str.replace(' ', '_', regex=False)
x.columns = x.columns.str.replace('%', '', regex=False)

In [39]:
x

,Geography,Unit_Sales,Product,Unit_Sales_No_Merch,Unit_Sales_Any_Merch,Unit_Sales_Price_Reductions_Only,Price_per_Unit,Price_per_Unit_No_Merch,Price_per_Unit_Any_Merch,Price_per_Unit_Price_Reductions_Only,year,Flavor,Brand,Product_No_Merch,Product_Any_Merch,Price_Change_
36805,California - Standard - Multi Outlet + Conv,232.985395,AMYS FROZEN BLACK BEAN MEAT SUBSTITUTE BURGER ...,229.985395,3.000000,3.000000,6.864690,6.889274,4.980000,4.980000,2020,BLACK BEAN,OTHERS,0.974575,0.012713,-0.277137
36806,California - Standard - Multi Outlet + Conv,313.706756,AMYS FROZEN CALIFORNIA VEGGIE MEAT SUBSTITUTE ...,302.807146,10.899610,7.039762,6.522935,6.555666,5.613643,5.407289,2020,REGULAR,OTHERS,0.944070,0.033982,-0.143696
36807,California - Standard - Multi Outlet + Conv,283.551283,AMYS FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE B...,279.691435,3.859848,53.850315,6.376719,6.382056,5.990000,4.179696,2020,REGULAR,OTHERS,0.828957,0.011440,-0.061431
36808,California - Standard - Multi Outlet + Conv,409.947925,AMYS FROZEN REGULAR MEAT SUBSTITUTE BURGER 16 ...,403.515004,6.432921,3.859689,8.129029,8.126399,8.294005,8.096667,2020,REGULAR,OTHERS,0.975127,0.015546,0.020625
36809,California - Standard - Multi Outlet + Conv,273.429006,AMYS FROZEN REGULAR MEAT SUBSTITUTE LINK 6 OZ ...,266.691106,6.737900,4.164668,6.694234,6.718803,5.721792,5.432500,2020,REGULAR,OTHERS,0.960725,0.024273,-0.148391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991775,West - Standard - Multi Outlet + Conv,23.790567,UPTONS NATURALS REFRIGERATED REGULAR MEAT SUBS...,23.790567,69.041003,53.850315,6.513050,6.513050,4.282196,4.179696,2024,REGULAR,OTHERS,0.162192,0.470685,-0.342521
991776,West - Standard - Multi Outlet + Conv,107.773600,UPTONS NATURALS REFRIGERATED TRADITIONAL SEITA...,73.564285,34.209315,34.209315,5.942539,6.545713,4.645463,4.645463,2024,REGULAR,OTHERS,0.518121,0.240940,-0.290305
991777,West - Standard - Multi Outlet + Conv,1240.627818,FRANKLIN FARMS REFRIGERATED CHICKEN PLANT BASE...,1110.825151,129.802667,100.438719,6.160735,6.155361,6.206728,6.174725,2024,CHICKEN,OTHERS,0.828315,0.096791,0.008345
991778,West - Standard - Multi Outlet + Conv,39.020600,LIGHTLIFE SMART DELI REFRIGERATED TURKEY POULT...,12.322003,26.698597,26.698597,3.550733,3.682327,3.490000,3.490000,2024,REGULAR,OTHERS,0.187495,0.406253,-0.052230


In [43]:
import statsmodels.api as sm
model = sm.OLS.from_formula('Unit_Sales ~ Brand + Geography +C(year)+ Price_Change_ + Price_Change_:Geography + Price_Change_:Brand+C(year):Brand', x).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     2854.
Date:                Sun, 28 Apr 2024   Prob (F-statistic):               0.00
Time:                        17:03:49   Log-Likelihood:            -7.6452e+06
No. Observations:              824367   AIC:                         1.529e+07
Df Residuals:                  824293   BIC:                         1.529e+07
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
                                                                                coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [41]:
model1 = sm.OLS.from_formula('Unit_Sales ~ Brand + Geography + Flavor + Price_per_Unit_Any_Merch + Price_per_Unit_Any_Merch:Geography', x).fit()

# Print the summary of the regression
print(model1.summary())

NameError: name 'sm' is not defined

In [ ]:
Unit sales = Anymerch%  + Brand + Anymerch%:Brand # second model

SyntaxError: invalid syntax (251765226.py, line 1)

In [42]:
x

,Geography,Unit_Sales,Product,Unit_Sales_No_Merch,Unit_Sales_Any_Merch,Unit_Sales_Price_Reductions_Only,Price_per_Unit,Price_per_Unit_No_Merch,Price_per_Unit_Any_Merch,Price_per_Unit_Price_Reductions_Only,year,Flavor,Brand,Product_No_Merch,Product_Any_Merch,Price_Change_
36805,California - Standard - Multi Outlet + Conv,232.985395,AMYS FROZEN BLACK BEAN MEAT SUBSTITUTE BURGER ...,229.985395,3.000000,3.000000,6.864690,6.889274,4.980000,4.980000,2020,BLACK BEAN,OTHERS,0.974575,0.012713,-0.277137
36806,California - Standard - Multi Outlet + Conv,313.706756,AMYS FROZEN CALIFORNIA VEGGIE MEAT SUBSTITUTE ...,302.807146,10.899610,7.039762,6.522935,6.555666,5.613643,5.407289,2020,REGULAR,OTHERS,0.944070,0.033982,-0.143696
36807,California - Standard - Multi Outlet + Conv,283.551283,AMYS FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE B...,279.691435,3.859848,53.850315,6.376719,6.382056,5.990000,4.179696,2020,REGULAR,OTHERS,0.828957,0.011440,-0.061431
36808,California - Standard - Multi Outlet + Conv,409.947925,AMYS FROZEN REGULAR MEAT SUBSTITUTE BURGER 16 ...,403.515004,6.432921,3.859689,8.129029,8.126399,8.294005,8.096667,2020,REGULAR,OTHERS,0.975127,0.015546,0.020625
36809,California - Standard - Multi Outlet + Conv,273.429006,AMYS FROZEN REGULAR MEAT SUBSTITUTE LINK 6 OZ ...,266.691106,6.737900,4.164668,6.694234,6.718803,5.721792,5.432500,2020,REGULAR,OTHERS,0.960725,0.024273,-0.148391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991775,West - Standard - Multi Outlet + Conv,23.790567,UPTONS NATURALS REFRIGERATED REGULAR MEAT SUBS...,23.790567,69.041003,53.850315,6.513050,6.513050,4.282196,4.179696,2024,REGULAR,OTHERS,0.162192,0.470685,-0.342521
991776,West - Standard - Multi Outlet + Conv,107.773600,UPTONS NATURALS REFRIGERATED TRADITIONAL SEITA...,73.564285,34.209315,34.209315,5.942539,6.545713,4.645463,4.645463,2024,REGULAR,OTHERS,0.518121,0.240940,-0.290305
991777,West - Standard - Multi Outlet + Conv,1240.627818,FRANKLIN FARMS REFRIGERATED CHICKEN PLANT BASE...,1110.825151,129.802667,100.438719,6.160735,6.155361,6.206728,6.174725,2024,CHICKEN,OTHERS,0.828315,0.096791,0.008345
991778,West - Standard - Multi Outlet + Conv,39.020600,LIGHTLIFE SMART DELI REFRIGERATED TURKEY POULT...,12.322003,26.698597,26.698597,3.550733,3.682327,3.490000,3.490000,2024,REGULAR,OTHERS,0.187495,0.406253,-0.052230


In [44]:
model2 = sm.OLS.from_formula('Price_Change_ ~ Product_Any_Merch+Brand+Brand:Product_Any_Merch', x).fit()

# Print the summary of the regression
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:          Price_Change_   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     2045.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        19:44:08   Log-Likelihood:            -1.3558e+05
No. Observations:              824367   AIC:                         2.712e+05
Df Residuals:                  824347   BIC:                         2.714e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [45]:
model3 = sm.OLS.from_formula('Price_Change_ ~ Product_No_Merch+Brand+Brand:Product_No_Merch', x).fit()

# Print the summary of the regression
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:          Price_Change_   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     1959.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        19:44:13   Log-Likelihood:            -1.3637e+05
No. Observations:              824367   AIC:                         2.728e+05
Df Residuals:                  824347   BIC:                         2.730e+05
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

In [46]:
model4 = sm.OLS.from_formula('Unit_Sales ~ Brand + C(year)+ Price_Change_ ', x).fit()

# Print the summary of the regression
print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                 1.156e+04
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        19:44:19   Log-Likelihood:            -7.6642e+06
No. Observations:              824367   AIC:                         1.533e+07
Df Residuals:                  824352   BIC:                         1.533e+07
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               4878

In [55]:
model5 = sm.OLS.from_formula('Price_Change_ ~ Brand + Product_No_Merch+ Product_Any_Merch ', x).fit()

# Print the summary of the regression
print(model5.summary())

                            OLS Regression Results                            
Dep. Variable:          Price_Change_   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     3068.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        19:58:32   Log-Likelihood:            -1.3803e+05
No. Observations:              824367   AIC:                         2.761e+05
Df Residuals:                  824355   BIC:                         2.762e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0

In [57]:
model6 = sm.OLS.from_formula('Price_Change_ ~ Brand + Product_No_Merch+ Product_Any_Merch+ Brand:Product_No_Merch+Brand:Product_Any_Merch', x).fit()

# Print the summary of the regression
print(model6.summary())

                            OLS Regression Results                            
Dep. Variable:          Price_Change_   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     1429.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:01:49   Log-Likelihood:            -1.3435e+05
No. Observations:              824367   AIC:                         2.688e+05
Df Residuals:                  824337   BIC:                         2.691e+05
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [63]:
model7 = sm.OLS.from_formula('Price_Change_ ~ Geography + Product_No_Merch+ Product_Any_Merch+ Geography:Product_No_Merch+Geography:Product_Any_Merch', x).fit()

# Print the summary of the regression
print(model7.summary())

                            OLS Regression Results                            
Dep. Variable:          Price_Change_   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     278.8
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:07:33   Log-Likelihood:            -1.5137e+05
No. Observations:              824367   AIC:                         3.028e+05
Df Residuals:                  824343   BIC:                         3.031e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                                                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [65]:
model8 = sm.OLS.from_formula('Price_Change_ ~ Product_No_Merch+ Product_Any_Merch', x).fit()

# Print the summary of the regression
print(model8.summary())

                            OLS Regression Results                            
Dep. Variable:          Price_Change_   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     514.9
Date:                Sat, 27 Apr 2024   Prob (F-statistic):          3.45e-224
Time:                        20:11:12   Log-Likelihood:            -1.5405e+05
No. Observations:              824367   AIC:                         3.081e+05
Df Residuals:                  824364   BIC:                         3.081e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.2263      0.00

In [69]:
model9 = sm.OLS.from_formula('Unit_Sales~ Product_No_Merch+ Product_Any_Merch+Brand+Brand:Product_No_Merch+Brand:Product_Any_Merch', x).fit()

# Print the summary of the regression
print(model9.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     7437.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:13:48   Log-Likelihood:            -7.6422e+06
No. Observations:              824367   AIC:                         1.528e+07
Df Residuals:                  824337   BIC:                         1.528e+07
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [77]:
model10 = sm.OLS.from_formula('Unit_Sales~ Product_No_Merch+Brand+Brand:Product_No_Merch', x).fit()

# Print the summary of the regression
print(model10.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.180
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     9555.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:18:07   Log-Likelihood:            -7.6560e+06
No. Observations:              824367   AIC:                         1.531e+07
Df Residuals:                  824347   BIC:                         1.531e+07
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

In [75]:
model11 = sm.OLS.from_formula('Unit_Sales~ Product_Any_Merch+Brand+Brand:Product_Any_Merch', x).fit()

# Print the summary of the regression
print(model11.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     9730.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:17:42   Log-Likelihood:            -7.6546e+06
No. Observations:              824367   AIC:                         1.531e+07
Df Residuals:                  824347   BIC:                         1.531e+07
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [79]:
x

,Geography,Unit_Sales,Product,Unit_Sales_No_Merch,Unit_Sales_Any_Merch,Unit_Sales_Price_Reductions_Only,Price_per_Unit,Price_per_Unit_No_Merch,Price_per_Unit_Any_Merch,Price_per_Unit_Price_Reductions_Only,year,Flavor,Brand,Product_No_Merch,Product_Any_Merch,Price_Change_
36805,California - Standard - Multi Outlet + Conv,232.985395,AMYS FROZEN BLACK BEAN MEAT SUBSTITUTE BURGER ...,229.985395,3.000000,3.000000,6.864690,6.889274,4.980000,4.980000,2020,BLACK BEAN,OTHERS,0.974575,0.012713,-0.277137
36806,California - Standard - Multi Outlet + Conv,313.706756,AMYS FROZEN CALIFORNIA VEGGIE MEAT SUBSTITUTE ...,302.807146,10.899610,7.039762,6.522935,6.555666,5.613643,5.407289,2020,REGULAR,OTHERS,0.944070,0.033982,-0.143696
36807,California - Standard - Multi Outlet + Conv,283.551283,AMYS FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE B...,279.691435,3.859848,53.850315,6.376719,6.382056,5.990000,4.179696,2020,REGULAR,OTHERS,0.828957,0.011440,-0.061431
36808,California - Standard - Multi Outlet + Conv,409.947925,AMYS FROZEN REGULAR MEAT SUBSTITUTE BURGER 16 ...,403.515004,6.432921,3.859689,8.129029,8.126399,8.294005,8.096667,2020,REGULAR,OTHERS,0.975127,0.015546,0.020625
36809,California - Standard - Multi Outlet + Conv,273.429006,AMYS FROZEN REGULAR MEAT SUBSTITUTE LINK 6 OZ ...,266.691106,6.737900,4.164668,6.694234,6.718803,5.721792,5.432500,2020,REGULAR,OTHERS,0.960725,0.024273,-0.148391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991775,West - Standard - Multi Outlet + Conv,23.790567,UPTONS NATURALS REFRIGERATED REGULAR MEAT SUBS...,23.790567,69.041003,53.850315,6.513050,6.513050,4.282196,4.179696,2024,REGULAR,OTHERS,0.162192,0.470685,-0.342521
991776,West - Standard - Multi Outlet + Conv,107.773600,UPTONS NATURALS REFRIGERATED TRADITIONAL SEITA...,73.564285,34.209315,34.209315,5.942539,6.545713,4.645463,4.645463,2024,REGULAR,OTHERS,0.518121,0.240940,-0.290305
991777,West - Standard - Multi Outlet + Conv,1240.627818,FRANKLIN FARMS REFRIGERATED CHICKEN PLANT BASE...,1110.825151,129.802667,100.438719,6.160735,6.155361,6.206728,6.174725,2024,CHICKEN,OTHERS,0.828315,0.096791,0.008345
991778,West - Standard - Multi Outlet + Conv,39.020600,LIGHTLIFE SMART DELI REFRIGERATED TURKEY POULT...,12.322003,26.698597,26.698597,3.550733,3.682327,3.490000,3.490000,2024,REGULAR,OTHERS,0.187495,0.406253,-0.052230


In [85]:
model12 = sm.OLS.from_formula('Unit_Sales~ Price_per_Unit_No_Merch+Price_per_Unit_Any_Merch+Price_per_Unit_Price_Reductions_Only', x).fit()

# Print the summary of the regression
print(model12.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     2050.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:30:34   Log-Likelihood:            -7.7350e+06
No. Observations:              824367   AIC:                         1.547e+07
Df Residuals:                  824363   BIC:                         1.547e+07
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [87]:
model13 = sm.OLS.from_formula('Unit_Sales~ Price_per_Unit_No_Merch+Price_per_Unit_Any_Merch+Price_per_Unit_Price_Reductions_Only+Geography:Price_per_Unit_Price_Reductions_Only+Geography', x).fit()

# Print the summary of the regression
print(model13.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     1016.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:32:21   Log-Likelihood:            -7.7295e+06
No. Observations:              824367   AIC:                         1.546e+07
Df Residuals:                  824349   BIC:                         1.546e+07
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------

In [91]:
model14 = sm.OLS.from_formula('Unit_Sales~ Price_per_Unit_Any_Merch+Price_per_Unit_Price_Reductions_Only+Brand:Price_per_Unit_Price_Reductions_Only+Brand+Brand:Price_per_Unit_Any_Merch', x).fit()

# Print the summary of the regression
print(model14.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     5865.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:36:30   Log-Likelihood:            -7.6607e+06
No. Observations:              824367   AIC:                         1.532e+07
Df Residuals:                  824337   BIC:                         1.532e+07
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [93]:
model15 = sm.OLS.from_formula('Unit_Sales~ Price_per_Unit_Any_Merch+Price_per_Unit_Price_Reductions_Only+Flavor:Price_per_Unit_Price_Reductions_Only+Flavor+Flavor:Price_per_Unit_Any_Merch', x).fit()

# Print the summary of the regression
print(model15.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     599.5
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:39:50   Log-Likelihood:            -7.7303e+06
No. Observations:              824367   AIC:                         1.546e+07
Df Residuals:                  824340   BIC:                         1.546e+07
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------

In [95]:
model16 = sm.OLS.from_formula('Unit_Sales~ Price_per_Unit_No_Merch+Flavor+Flavor:Price_per_Unit_No_Merch', x).fit()

# Print the summary of the regression
print(model16.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     958.4
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:42:02   Log-Likelihood:            -7.7300e+06
No. Observations:              824367   AIC:                         1.546e+07
Df Residuals:                  824349   BIC:                         1.546e+07
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [97]:
x

,Geography,Unit_Sales,Product,Unit_Sales_No_Merch,Unit_Sales_Any_Merch,Unit_Sales_Price_Reductions_Only,Price_per_Unit,Price_per_Unit_No_Merch,Price_per_Unit_Any_Merch,Price_per_Unit_Price_Reductions_Only,year,Flavor,Brand,Product_No_Merch,Product_Any_Merch,Price_Change_
36805,California - Standard - Multi Outlet + Conv,232.985395,AMYS FROZEN BLACK BEAN MEAT SUBSTITUTE BURGER ...,229.985395,3.000000,3.000000,6.864690,6.889274,4.980000,4.980000,2020,BLACK BEAN,OTHERS,0.974575,0.012713,-0.277137
36806,California - Standard - Multi Outlet + Conv,313.706756,AMYS FROZEN CALIFORNIA VEGGIE MEAT SUBSTITUTE ...,302.807146,10.899610,7.039762,6.522935,6.555666,5.613643,5.407289,2020,REGULAR,OTHERS,0.944070,0.033982,-0.143696
36807,California - Standard - Multi Outlet + Conv,283.551283,AMYS FROZEN GARDEN VEGETABLE MEAT SUBSTITUTE B...,279.691435,3.859848,53.850315,6.376719,6.382056,5.990000,4.179696,2020,REGULAR,OTHERS,0.828957,0.011440,-0.061431
36808,California - Standard - Multi Outlet + Conv,409.947925,AMYS FROZEN REGULAR MEAT SUBSTITUTE BURGER 16 ...,403.515004,6.432921,3.859689,8.129029,8.126399,8.294005,8.096667,2020,REGULAR,OTHERS,0.975127,0.015546,0.020625
36809,California - Standard - Multi Outlet + Conv,273.429006,AMYS FROZEN REGULAR MEAT SUBSTITUTE LINK 6 OZ ...,266.691106,6.737900,4.164668,6.694234,6.718803,5.721792,5.432500,2020,REGULAR,OTHERS,0.960725,0.024273,-0.148391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991775,West - Standard - Multi Outlet + Conv,23.790567,UPTONS NATURALS REFRIGERATED REGULAR MEAT SUBS...,23.790567,69.041003,53.850315,6.513050,6.513050,4.282196,4.179696,2024,REGULAR,OTHERS,0.162192,0.470685,-0.342521
991776,West - Standard - Multi Outlet + Conv,107.773600,UPTONS NATURALS REFRIGERATED TRADITIONAL SEITA...,73.564285,34.209315,34.209315,5.942539,6.545713,4.645463,4.645463,2024,REGULAR,OTHERS,0.518121,0.240940,-0.290305
991777,West - Standard - Multi Outlet + Conv,1240.627818,FRANKLIN FARMS REFRIGERATED CHICKEN PLANT BASE...,1110.825151,129.802667,100.438719,6.160735,6.155361,6.206728,6.174725,2024,CHICKEN,OTHERS,0.828315,0.096791,0.008345
991778,West - Standard - Multi Outlet + Conv,39.020600,LIGHTLIFE SMART DELI REFRIGERATED TURKEY POULT...,12.322003,26.698597,26.698597,3.550733,3.682327,3.490000,3.490000,2024,REGULAR,OTHERS,0.187495,0.406253,-0.052230


In [99]:
model17 = sm.OLS.from_formula('Unit_Sales~ Price_per_Unit_No_Merch+Flavor+Geography+Flavor:Price_per_Unit_No_Merch:Geography', x).fit()

# Print the summary of the regression
print(model17.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     330.2
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:50:08   Log-Likelihood:            -7.7239e+06
No. Observations:              824367   AIC:                         1.545e+07
Df Residuals:                  824279   BIC:                         1.545e+07
Df Model:                          87                                         
Covariance Type:            nonrobust                                         
                                                                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

In [105]:
model18 = sm.OLS.from_formula('Unit_Sales~ Geography+Brand+Price_per_Unit_Price_Reductions_Only+Price_per_Unit_Any_Merch+Geography:Price_per_Unit_Any_Merch:Brand+Geography:Price_per_Unit_Price_Reductions_Only:Brand', x).fit()

# Print the summary of the regression
print(model18.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     1228.
Date:                Sat, 27 Apr 2024   Prob (F-statistic):               0.00
Time:                        21:10:54   Log-Likelihood:            -7.6421e+06
No. Observations:              824367   AIC:                         1.528e+07
Df Residuals:                  824190   BIC:                         1.529e+07
Df Model:                         176                                         
Covariance Type:            nonrobust                                         
                                                                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------

In [107]:
model19 = sm.OLS.from_formula('Unit_Sales~ Brand+Flavor+Price_per_Unit_Any_Merch+Price_per_Unit_No_Merch+Brand:Flavor:Price_per_Unit_Any_Merch+Brand:Flavor:Price_per_Unit_No_Merch', x).fit()

# Print the summary of the regression
print(model19.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     1921.
Date:                Sun, 28 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:19:33   Log-Likelihood:            -7.6327e+06
No. Observations:              824367   AIC:                         1.527e+07
Df Residuals:                  824241   BIC:                         1.527e+07
Df Model:                         125                                         
Covariance Type:            nonrobust                                         
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [117]:
model20 = sm.OLS.from_formula('Unit_Sales~ Brand+Flavor+Price_per_Unit_Any_Merch+Price_per_Unit_No_Merch+Flavor:Brand:Price_per_Unit_Any_Merch+Flavor:Brand:Price_per_Unit_No_Merch', x).fit()

# Print the summary of the regression
print(model20.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     1891.
Date:                Sun, 28 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:32:52   Log-Likelihood:            -7.6327e+06
No. Observations:              824367   AIC:                         1.527e+07
Df Residuals:                  824239   BIC:                         1.527e+07
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [115]:
model21 = sm.OLS.from_formula('Unit_Sales~ Brand+Flavor+Price_per_Unit_Any_Merch+Brand:Price_per_Unit_Any_Merch:Flavor', x).fit()

# Print the summary of the regression
print(model21.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.206
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     2929.
Date:                Sun, 28 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:27:05   Log-Likelihood:            -7.6430e+06
No. Observations:              824367   AIC:                         1.529e+07
Df Residuals:                  824293   BIC:                         1.529e+07
Df Model:                          73                                         
Covariance Type:            nonrobust                                         
                                                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

Flavor Focused Model = Unit Sales = Brand + Flavor + Price Any Merch + Price No Merch + Price Reductions + (Brand:Flavor:PriceAnyMerch) + (Brand:Flavor:PriceNoMerch) + (Brand:Flavor:PriceReductions)

In [53]:
Flavor_m = sm.OLS.from_formula('Unit_Sales~ Brand+Flavor+Price_per_Unit_Any_Merch+Price_per_Unit_No_Merch+Price_per_Unit_Price_Reductions_Only+Flavor:C(Brand):Price_per_Unit_Any_Merch+Flavor:C(Brand):Price_per_Unit_No_Merch+Flavor:C(Brand):Price_per_Unit_Price_Reductions_Only', x).fit()

# Print the summary of the regression
print(Flavor_m.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.230
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     1384.
Date:                Mon, 29 Apr 2024   Prob (F-statistic):               0.00
Time:                        22:54:35   Log-Likelihood:            -7.6303e+06
No. Observations:              824367   AIC:                         1.526e+07
Df Residuals:                  824188   BIC:                         1.526e+07
Df Model:                         178                                         
Covariance Type:            nonrobust                                         
                                                                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------

In [47]:
Geography_m = sm.OLS.from_formula('Unit_Sales~ Brand+Geography+Price_per_Unit_Any_Merch+Price_per_Unit_No_Merch+Price_per_Unit_Price_Reductions_Only+Geography:Brand:Price_per_Unit_Any_Merch+Geography:Brand:Price_per_Unit_No_Merch+Geography:Brand:Price_per_Unit_Price_Reductions_Only', x).fit()

# Print the summary of the regression
print(Geography_m.summary())

                            OLS Regression Results                            
Dep. Variable:             Unit_Sales   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     952.4
Date:                Sun, 28 Apr 2024   Prob (F-statistic):               0.00
Time:                        17:37:41   Log-Likelihood:            -7.6312e+06
No. Observations:              824367   AIC:                         1.526e+07
Df Residuals:                  824110   BIC:                         1.527e+07
Df Model:                         256                                         
Covariance Type:            nonrobust                                         
                                                                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------